In [1]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.layers import Conv1D, SpatialDropout1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as ply
%matplotlib inline

C:\Amrendra\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = "D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\train_E6oV3lV.csv"
test = 'D:\\My Personal Documents\\Learnings\\Data Science\\Data Sets\\Twiter Sentiment Analysis\\test_tweets_anuFYb8.csv'
tweets = pd.read_csv(file)
test_tweets=pd.read_csv(test)

In [3]:
tweets1=tweets[tweets.label==1]
tweets0=tweets[tweets.label==0]
print(tweets1.count())
print(tweets0.count())

id       2242
label    2242
tweet    2242
dtype: int64
id       29720
label    29720
tweet    29720
dtype: int64


In [4]:
tweets0_undersample=tweets0.sample(frac=0.3)

In [5]:
tweets=tweets1
tweets=tweets.append(tweets0_undersample)
label=tweets.label

In [6]:
tweets.tweet.count()

11158

In [7]:
tweets=tweets.append(test_tweets)

C:\Amrendra\Anaconda\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [8]:
tweets.groupby(tweets.label).tweet.count()

label
0.0    8916
1.0    2242
Name: tweet, dtype: int64

In [9]:
cleaned_tweet = tweets.tweet.str.replace("[^a-zA-Z]", " ")

In [10]:
cleaned_tweet=cleaned_tweet.apply(lambda x :" ".join(w.lower() for w in x.split() if  w != 'user' ))

In [11]:
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer

In [12]:
stem=PorterStemmer()

In [13]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( stem.stem(w) for w in x.split()))

In [14]:
cleaned_tweet = cleaned_tweet.apply(lambda x : " ".join( w for w in x.split()))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf = TfidfVectorizer(max_features=3000,stop_words='english')

In [17]:
bow=tfidf.fit_transform(cleaned_tweet)

In [18]:
train_bow = bow[:11158,:]
test_bow = bow[11158:,:]

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train, y_test = train_test_split(train_bow,label,test_size=0.2,random_state=3)

In [21]:
from sklearn import svm
model_svm= svm.SVC(kernel='rbf', C=1,gamma=1)
model_svm.fit(x_train,y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
pred_svm=model_svm.predict(x_test)

In [23]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [24]:
metrics.accuracy_score(y_test,pred_svm)

0.9126344086021505

In [25]:
test_pred_svm= model_svm.predict(test_bow)

In [26]:
from sklearn.naive_bayes import MultinomialNB
model_NB= MultinomialNB(0.25)
model_NB.fit(x_train,y_train)
pred_NB=model_NB.predict(x_test)
from sklearn import metrics
from sklearn.metrics import confusion_matrix
metrics.accuracy_score(y_test,pred_NB)

0.9099462365591398

In [27]:
test_pred_NB= model_NB.predict(test_bow)

In [28]:
from sklearn.linear_model import LogisticRegression
model_LR= LogisticRegression()
model_LR.fit(x_train,y_train)
test_pred_LR= model_LR.predict(test_bow)

C:\Amrendra\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model_xgb = XGBClassifier( nthread=4, n_estimators=200, learning_rate=0.2,
       colsample_bytree=0.85,
        max_depth=50,
        reg_lambda=1,
min_split_gain=0.0222415)
model_xgb.fit(x_train,y_train)
pred_xgb=model_xgb.predict(x_test)

from sklearn import metrics
from sklearn.metrics import confusion_matrix

metrics.accuracy_score(y_test,pred_xgb)

0.8991935483870968

In [30]:
test_pred_xgb= model_xgb.predict(test_bow)

In [31]:
from keras.callbacks import ModelCheckpoint
import os
output_dir = 'model_tweet_output1/dense'
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [32]:
x_train,x_test,y_train, y_test = train_test_split(train_bow,label,test_size=0.2,random_state=3)

model_mlp_tfidf = Sequential()
model_mlp_tfidf.add(Dense(1000,activation='relu', input_shape=(3000,)))
model_mlp_tfidf.add(Dropout(0.2))
model_mlp_tfidf.add(Dense(1,activation='sigmoid'))
model_mlp_tfidf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_mlp_tfidf.fit(x_train,y_train,epochs=3,verbose=1, validation_data=(x_test,y_test),callbacks=[modelcheckpoint])

Train on 8926 samples, validate on 2232 samples
Epoch 1/3
8926/8926 [==============================] - 19s 2ms/step - loss: 0.3241 - acc: 0.8605 - val_loss: 0.2268 - val_acc: 0.9108
Epoch 2/3
8926/8926 [==============================] - 18s 2ms/step - loss: 0.1539 - acc: 0.9386 - val_loss: 0.2320 - val_acc: 0.9091
Epoch 3/3
8926/8926 [==============================] - 20s 2ms/step - loss: 0.0995 - acc: 0.9639 - val_loss: 0.2640 - val_acc: 0.8992


In [33]:
model_mlp_tfidf.load_weights(output_dir+'/weights.03.hdf5')
test_pred_mlp_tfidf=model_mlp_tfidf.predict_classes(test_bow)

In [34]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweet)
sequences = tokenizer.texts_to_sequences(cleaned_tweet)

from keras.preprocessing.sequence import pad_sequences
word_index=pad_sequences(sequences, maxlen=40, padding = 'pre', truncating='pre')

train = word_index[:11158,:]
test = word_index[11158:,:]

from sklearn.model_selection import train_test_split

x_train,x_test,y_train, y_test = train_test_split(train,label,test_size=0.2,random_state=3, stratify=label)

In [40]:
output_dir = 'model_tweet_output/LSTM'
epochs = 5
batch_size = 64
n_dim = 64
n_unique_words = 35000
n_words_to_skip = 5
max_review_length = 40
pad_type = trunc_type = 'pre'

model_lstm = Sequential()
model_lstm.add(Embedding(n_unique_words,n_dim, input_length=max_review_length))
model_lstm.add(LSTM(30,return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(30))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_lstm.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test), callbacks=[modelcheckpoint])

Train on 8926 samples, validate on 2232 samples
Epoch 1/5
8926/8926 [==============================] - 15s 2ms/step - loss: 0.4151 - acc: 0.8306 - val_loss: 0.2631 - val_acc: 0.8893
Epoch 2/5
8926/8926 [==============================] - 15s 2ms/step - loss: 0.1687 - acc: 0.9361 - val_loss: 0.2286 - val_acc: 0.9113
Epoch 3/5
8926/8926 [==============================] - 13s 1ms/step - loss: 0.0870 - acc: 0.9724 - val_loss: 0.2512 - val_acc: 0.9122
Epoch 4/5
8926/8926 [==============================] - 13s 1ms/step - loss: 0.0509 - acc: 0.9842 - val_loss: 0.2932 - val_acc: 0.9091
Epoch 5/5
8926/8926 [==============================] - 14s 2ms/step - loss: 0.0284 - acc: 0.9917 - val_loss: 0.3479 - val_acc: 0.9095


In [41]:
model_lstm.load_weights(output_dir+'/weights.03.hdf5')
test_pred_lstm = model_lstm.predict_classes(test)

In [43]:
output_dir = 'model_tweet_output/cnn'
epochs = 5
batch_size = 128
n_dim = 128
n_unique_words = 35000
n_words_to_skip = 5
max_review_length = 40
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_dense = 256
dropout = 0.2

n_conv = 256
k_conv = 3

In [44]:
model_cnn = Sequential()
model_cnn.add(Embedding(n_unique_words,n_dim, input_length=max_review_length))
model_cnn.add(SpatialDropout1D(drop_embed))
model_cnn.add(Conv1D(n_conv, k_conv, activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(n_dense,activation='relu'))
model_cnn.add(Dropout(dropout))
model_cnn.add(Dense(1, activation='sigmoid'))

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_cnn.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test,y_test), callbacks=[modelcheckpoint])

Train on 8926 samples, validate on 2232 samples
Epoch 1/5
8926/8926 [==============================] - 145s 16ms/step - loss: 0.4565 - acc: 0.7946 - val_loss: 0.3393 - val_acc: 0.8517
Epoch 2/5
8926/8926 [==============================] - 144s 16ms/step - loss: 0.2179 - acc: 0.9199 - val_loss: 0.2313 - val_acc: 0.9073
Epoch 3/5
8926/8926 [==============================] - 142s 16ms/step - loss: 0.0895 - acc: 0.9715 - val_loss: 0.2616 - val_acc: 0.9028
Epoch 4/5
8926/8926 [==============================] - 143s 16ms/step - loss: 0.0362 - acc: 0.9901 - val_loss: 0.3117 - val_acc: 0.9082
Epoch 5/5
8926/8926 [==============================] - 145s 16ms/step - loss: 0.0151 - acc: 0.9963 - val_loss: 0.3694 - val_acc: 0.9019


In [45]:
model_cnn.load_weights(output_dir+'/weights.02.hdf5')
test_pred_cnn = model_cnn.predict_classes(test)

In [46]:
import numpy as np

In [47]:
test_pred_NB = pd.DataFrame(test_pred_NB)
test_pred_svm = pd.DataFrame(test_pred_svm)
test_pred_LR = pd.DataFrame(test_pred_LR)
test_pred_xgb = pd.DataFrame(test_pred_xgb)
test_pred_lstm = pd.DataFrame(test_pred_lstm)
test_pred_cnn = pd.DataFrame(test_pred_cnn)
test_pred_mlp_tfidf = pd.DataFrame(test_pred_mlp_tfidf)

In [48]:
output = pd.concat([test_pred_NB,test_pred_svm,test_pred_LR,test_pred_xgb,test_pred_lstm,test_pred_cnn, test_pred_mlp_tfidf],axis=1)

In [49]:
output1=output.mode(axis=1)
output2=pd.concat([output1,test_pred_xgb,test_pred_mlp_tfidf,test_pred_NB,test_pred_lstm],axis=1).mode(axis=1)

In [50]:
final_output=pd.DataFrame(pd.concat([test_tweets.id,output2], axis=1))
final_output.columns=['id','label']
final_output.to_csv("submission.csv", index=False)